In [26]:
import pandas as pd
import json 
import os

In [27]:
directory = r'C:\Users\vinze\Desktop\Uni\DH\Forschun gsmethoden\corpus_finished'

In [28]:
df = pd.DataFrame()
failed_files = 0        # counter für except fall


for filename in os.listdir(directory):
    try:
        json_df = pd.read_json(directory + '/' + filename)
        json_df['line'] = json_df.index.str.split('.').str[1].astype(int)
        json_df = json_df.sort_values(by=['line'])
        df = pd.concat([df, json_df])
    except:
        print(filename)
        failed_files += 1

df['text'] = df['poem'].apply(pd.Series)['text']
df = df.reset_index(drop=True)

temp_list = []
for text in df['text']:
    temp_list.append(len(text.split()))

df = pd.concat([df, pd.DataFrame({'length':temp_list})], axis=1)


aus-duengerkind-i-15421.json
aus-duengerkind-ii-15422.json
brief-im-april-15724.json
der-pollenflug-15513.json
der-schiffsbaumeister-10090.json
der-stille-grund-11236.json
der-wasserwolf-13929.json
die-maulwuerfe-14378.json
donauwuerfel-vierter-wuerfel-15111.json
du-bist-eine-lilie-15524.json
es-muss-nochmals-darauf-hingewiesen-werden-14213.json
gartenweg-5380.json
herde-der-rede-v-381.json
hinterland-i-iv-3381.json
im-brennglas-13934.json
kara-burun-verlassenes-dorf-5665.json
kinderheim-5-13408.json
kinderheim-_-13402.json
kleine-unruhe-13979.json
koenigreich-der-verwaisten-bienenstoecke-15217.json
neumen-unwelt-korollar.json
postkarte-15723.json
silvae-gelichtet-4020.json
so-augenweide-achsen-doch-als-springe-und-trete-man-rueckwaerts-15456.json
sternengeschichte-450.json
svalbard-paem-15118.json
tierleben-0-4009.json
vor-jahren-fiel-es-mir-schwer-das-licht-im-bett-auszuknipsen-15423.json
wald-vogel-6046.json
zwangsmittel-12267.json


In [32]:
def get_statistics(df, gender):
    num_unique_titles = df['title'].nunique()
    num_unique_authors = df['author'].nunique()
    total_tokens = df['length'].sum()

    print(f"\nStatistik für das Geschlecht '{gender}':")
    print(f"Anzahl der einzigartigen Titel: {num_unique_titles}")
    print(f"Anzahl der einzigartigen Autoren: {num_unique_authors}")
    print(f"Gesamtzahl der Token: {total_tokens}")

# Get statistics for gender 'm'
df_m = df[df['gender'] == 'm']
get_statistics(df_m, 'm')

# Get statistics for gender 'w'
df_w = df[df['gender'] == 'w']
get_statistics(df_w, 'w')

# Get statistics for undefined gender
df_undefined = df[df['gender'] == '']
get_statistics(df_undefined, 'undefined')

print(f"\nAnzahl der Dateien, die nicht verarbeitet wurden: {failed_files}")


Statistik für das Geschlecht 'm':
Anzahl der einzigartigen Titel: 0
Anzahl der einzigartigen Autoren: 0
Gesamtzahl der Token: 0

Statistik für das Geschlecht 'w':
Anzahl der einzigartigen Titel: 0
Anzahl der einzigartigen Autoren: 0
Gesamtzahl der Token: 0

Statistik für das Geschlecht 'undefined':
Anzahl der einzigartigen Titel: 87
Anzahl der einzigartigen Autoren: 46
Gesamtzahl der Token: 10926

Anzahl der Dateien, die nicht verarbeitet wurden: 30


In [30]:
df

,title,gender,author,poem,line,text,length
0,alemannische-mangos-15731,,Marie T. Martin,{'text': 'Wenn die Zweige zum Boden geneigt si...,1,Wenn die Zweige zum Boden geneigt sind kannst du,9
1,alemannische-mangos-15731,,Marie T. Martin,{'text': 'auf den Baum klettern immer weiter s...,2,auf den Baum klettern immer weiter so wie man,9
2,alemannische-mangos-15731,,Marie T. Martin,{'text': 'die Leiter wegwirft auf der man steh...,3,die Leiter wegwirft auf der man steht wenn du ...,10
3,alemannische-mangos-15731,,Marie T. Martin,{'text': 'sind die klein geschrumpften Brombee...,4,sind die klein geschrumpften Brombeeren aus di...,8
4,alemannische-mangos-15731,,Marie T. Martin,{'text': 'alemannische Mangos die immer noch s...,5,alemannische Mangos die immer noch süß schmecken,7
...,...,...,...,...,...,...,...
1803,wyoming-13239,,Mirko Bonné,{'text': 'Wenn die Zuneigung zurückkehrte mit ...,12,Wenn die Zuneigung zurückkehrte mit den Fischen.,7
1804,wyoming-13239,,Mirko Bonné,"{'text': 'Wenn alle die Fragen wohin, wodurch,...",13,"Wenn alle die Fragen wohin, wodurch, wonach, w...",8
1805,wyoming-13239,,Mirko Bonné,{'text': 'weshalb und wann die Antworten erset...,14,weshalb und wann die Antworten ersetzten. Und ...,9
1806,wyoming-13239,,Mirko Bonné,"{'text': 'mit einem Mal bliebe. Hier, dort. In...",15,"mit einem Mal bliebe. Hier, dort. In Wyoming, ...",11
